## Underway data
Underway data provided by MI, its a research vessel data.<br>
**Key Points about data**
1. SST and Salinity readings is for the surface and are random points data as reading taken by ships.<br>
2. Data is been collected for every 10seconds/day (Temporal resolution).<br>
   But as our aim is daily resolution, averaging needed to be done for each day.
3. Underway data will the base for the data preparation for SST.<br>
4. Data covers for Ireland and around UK region, but we will use data only for the bounding box region,<br>
   **[-14, -5, 49, 56]**

**Reading the original data using pandas**<br>
To know the shape, datatype of each column etc about the data.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Load the data into a pandas dataframe
df=pd.read_csv('original_UW_data/UW_data.csv',names=["Datetime", "Lon", "Lat", "Temperature","Salinity"])

#Check the shape of dataframe
print('Shape of Original underway data:',df.shape)

#Check the data types of each column and make sure the datetime column is of the datetime format
print('Datatype of each column: ',df.dtypes)


**Drop Salinity Column**
As for now parameter of focus is SST.<br>

In [ ]:
#Dropping Salinity Column and saving a new file only for temperature
df=df.drop(columns='Salinity')
df.to_csv('original_UW_data/UW_tempdata.csv',index=None)

In [ ]:
#Reading the SST file
df=pd.read_csv('original_UW_data/UW_tempdata.csv')
df

In [ ]:
df.dtypes

**Datetime has object datatype, converting it to datetime64[ns].**<br>

In [ ]:
#Converting to datetime format from object to datetime64[ns]
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [ ]:
df.dtypes

In [ ]:
#Check the first five rows of the dataframe using the head() function
df.head()

### Missing data
Missing data is -999.0, there are 1232 missing data, we can drop these
Original data has (54585495, 4), removing missing data it should be (54584263,4)

In [ ]:
#Check the missing values
#print('Missing values:',df.isnull().sum())
m=df[df['Temperature']== -999.0]
m.shape
m

**Dropping the rows having -999.0 value.**

In [ ]:
# Drop rows where temperature is -999.0
df1 = df[df['Temperature'] != -999.0]

# Save the updated DataFrame
df1.to_csv("original_UW_data/UW_data_r.csv", index=None)

In [ ]:
#Read the csv file where -999.0 values is removed
df1=pd.read_csv("original_UW_data/UW_data_r.csv")
df1.shape

In [ ]:
df1.info()

In [ ]:
#Check for outliers in the temperature column using a boxplot
plt.figure(figsize=[18,10])
sns.set(style="darkgrid")
plt.rcParams.update({'font.size': 20, 'font.weight': 'bold'})
sns.boxplot(x=df1['Temperature'],width=0.5,linewidth=2,color='darkgoldenrod')
plt.xlabel('Temperature', fontsize=14, fontweight='bold')
plt.title('Boxplot of Temperature', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
#Plot the distribution of the temperature column using a histogram
plt.figure(figsize=[18,10])
plt.rcParams.update({'font.size': 20, 'font.weight': 'bold'})
plt.hist(df1['Temperature'], bins=20, color='darkgoldenrod')
plt.xlabel('Temperature', fontsize=14, fontweight='bold')
plt.ylabel('Count', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# Set font size and style
plt.figure(figsize=[18,10])
plt.rcParams.update({'font.size': 20, 'font.weight': 'bold'})
plt.plot(df1['Temperature'],color='darkgoldenrod')
plt.xlabel('Datetime', fontsize=14, fontweight='bold')
plt.ylabel('Temperature', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# Convert datetime column to datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Filter the data based only on a particular date
date_filter = df[df['Datetime'].dt.date == pd.to_datetime('1994-03-25').date()]

# Print the filtered data
print(date_filter)

In [ ]:
#The data is for the larger area than required for us within the bounding box 
#Next we will filter the data for the considered Ireland bounding box region
import cartopy.crs as ccrs
import cartopy as cart
%matplotlib inline


fig = plt.figure(figsize=[18,10])
ax = plt.axes(projection=ccrs.PlateCarree())

plt.scatter(df1.Lon, df1.Lat, s=10, color='darkgoldenrod', marker='*',
             transform=ccrs.PlateCarree())

ax.coastlines(resolution='50m')
ax.add_feature(cart.feature.LAND, zorder=100, edgecolor='k')
plt.ylabel ('Latitude')
plt.xlabel ('Longitude')

plt.show()

In [ ]:
#Making the change in the csv file and saving that change to the csv file,now the file stores

df1=df1[df1['Lon']>=-14]
df1=df1[df1['Lon']<=-5]
df1=df1[df1['Lat']<=56]
df1=df1[df1['Lat']>=49]
#df.shape

df1.to_csv('original_UW_data/UW_Data_r_Ireland.csv',index=None)#The file for only Ireland region with bounding box[-14,-5,49,56]

In [ ]:
import pandas as pd



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Define the longitude and latitude limits for your region of interest
lon_min, lon_max, lat_min, lat_max = -14, -5, 49, 56

# Define the name of your region of interest
region = 'My Region'

# Load the data into a pandas DataFrame
df = pd.read_csv('daily_averages.csv')

# Get the list of unique years and months in the data
years = df['year'].unique()
months = df['month'].unique()

# Loop over the years and months and create a plot for each combination
for year in years:
    for month in months:
        # Filter the data for the given year and month
        data = df[(df['year'] == year) & (df['month'] == month)]

        # Create a new figure with a single subplot
        fig, ax = plt.subplots(1, 1, figsize=(15, 10), subplot_kw={'projection': ccrs.PlateCarree()})

        # Define the projection and extent of the map
        proj = ccrs.PlateCarree()
        extent = [lon_min, lon_max, lat_min, lat_max]
        
        #Add ocean feature to the map
        ax.add_feature(cfeature.OCEAN, zorder=1)
        
        # Plot the temperature data on the map
        sc = ax.scatter(data['Lon'], data['Lat'], c=data['Temperature'], cmap='plasma', transform=proj,s=120)
        
        ########################################################
        ####################Annotation##########################
        # Add annotations with the date to each plotted data point
        #for i, row in data.iterrows():
            #ax.annotate('{:04d}-{:02d}-{:02d}'.format(int(row['year']), int(row['month']), int(row['day'])),
                        #xy=(row['Lon'], row['Lat']), xytext=(3, 3),
                        #textcoords='offset points', fontsize=9)
            
        # Add the land and ocean features to the map
        ax.coastlines(resolution='50m')
        ax.add_feature(cart.feature.LAND, zorder=100, edgecolor='k')

        # Add a colorbar to the map
        cbar = plt.colorbar(sc, ax=ax, shrink=0.9)
        cbar.set_label('Temperature (°C)',fontsize=14,fontweight='bold')

        # Set the title and extent of the map
        ax.set_title('{}-{}'.format(year, month),fontsize=14,fontweight='bold')
        ax.set_extent(extent)

        # Show the plot
        plt.show()

        # Close the figure
        #plt.close(fig)

In [ ]:
#Reading the SST file only for the considered bounding box region
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy as cart
import cartopy.feature as cfeature
import seaborn as sns
df1=pd.read_csv('original_UW_data/UW_data_r_Ireland.csv')

In [ ]:
#Original shape :(54585495, 4)
#Now filtering BB and -999.0(45710933, 4)
df1.shape

In [ ]:
#Filtering done for the BB
plt.figure(figsize=[18,10])
ax = plt.axes(projection=ccrs.PlateCarree())

plt.scatter(df1.Lon, df1.Lat, s=10, color='darkgoldenrod', marker='*',
             transform=ccrs.PlateCarree())

ax.coastlines(resolution='50m')
ax.add_feature(cart.feature.LAND, zorder=100, edgecolor='k')
plt.ylabel ('Latitude')
plt.xlabel ('Longitude')

plt.show()

In [ ]:

# convert datetime column to datetime format
df1['Datetime'] = pd.to_datetime(df1['Datetime'])

# extract year from datetime column
df1['Year'] = df1['Datetime'].dt.year

# create a list of years
years = list(df1['Year'].unique())

# set figure size
figsize=(15,10)

# create a loop to plot each year's data
for year in years:
    # filter the data by year
    data = df1[df1['Year'] == year]

    # set up the map projection
    ax = plt.axes(projection=ccrs.PlateCarree())

    #Add ocean feature to the map
    ax.add_feature(cfeature.OCEAN, zorder=1)

    # plot the temperature data
    plt.scatter(data['Lon'], data['Lat'], c=data['Temperature'], s=10, cmap='plasma', transform=ccrs.PlateCarree())

    # add coastlines to the map
    #ax.add_feature(cfeature.COASTLINE)
    ######################################
   
    ax.coastlines(resolution='50m')
    ax.add_feature(cart.feature.LAND, zorder=100, edgecolor='k')
    #############################################
    # set the colorbar
    cbar = plt.colorbar(shrink=0.5)
    cbar.ax.set_ylabel('Temperature (°C)', fontsize=14)

    # set the title
    plt.title(f'Temperature Data for {year}', fontsize=16, fontweight='bold')

    # set x and y axis labels
    plt.xlabel('Longitude', fontsize=14, fontweight='bold')
    plt.ylabel('Latitude', fontsize=14, fontweight='bold')

    # set the figure size
    plt.gcf().set_size_inches(figsize)

    # show the plot
    plt.show()

**Calculating daily averages over the temperature,longitude and latitude**<br>
**Creating new files holding the average for each day**<br>
**Data is from 1994-2022**<br>

In [ ]:

# Convert the 'datetime' column to a pandas datetime object
df1['Datetime'] = pd.to_datetime(df1['Datetime'])

# Group the data by year, month, and day
grouped = df1.groupby([df1['Datetime'].dt.year, df1['Datetime'].dt.month, df1['Datetime'].dt.day])

# Calculate the daily average of temperature, longitude, and latitude
daily_averages = grouped.agg({'Temperature': 'mean', 'Lon': 'mean', 'Lat': 'mean'})

# Rename the index to 'year', 'month', and 'day'
daily_averages.index.names = ['year', 'month', 'day']

# Reset the index to convert the grouped DataFrame to a regular DataFrame
daily_averages = daily_averages.reset_index()

# Print the resulting DataFrame
print(daily_averages)

In [ ]:
daily_averages=daily_averages.to_csv('daily_averages.csv',index=None)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Define the longitude and latitude limits for your region of interest
lon_min, lon_max, lat_min, lat_max = -14, -5, 49, 56

# Define the name of your region of interest
region = 'My Region'

# Load the data into a pandas DataFrame
df = pd.read_csv('daily_averages.csv')

# Get the list of unique years and months in the data
years = df['year'].unique()
months = df['month'].unique()

# Loop over the years and months and create a plot for each combination
for year in years:
    for month in months:
        # Filter the data for the given year and month
        data = df[(df['year'] == year) & (df['month'] == month)]

        # Create a new figure with a single subplot
        fig, ax = plt.subplots(1, 1, figsize=(15, 10), subplot_kw={'projection': ccrs.PlateCarree()})

        # Define the projection and extent of the map
        proj = ccrs.PlateCarree()
        extent = [lon_min, lon_max, lat_min, lat_max]
        
        #Add ocean feature to the map
        ax.add_feature(cfeature.OCEAN, zorder=1)
        
        # Plot the temperature data on the map
        sc = ax.scatter(data['Lon'], data['Lat'], c=data['Temperature'],marker='^', cmap='plasma', transform=proj,s=120)
        
        ########################################################
        ####################Annotation##########################
        # Add annotations with the date to each plotted data point
        #for i, row in data.iterrows():
            #ax.annotate('{:04d}-{:02d}-{:02d}'.format(int(row['year']), int(row['month']), int(row['day'])),
                        #xy=(row['Lon'], row['Lat']), xytext=(3, 3),
                        #textcoords='offset points', fontsize=9)
            
        # Add the land and ocean features to the map
        ax.coastlines(resolution='50m')
        ax.add_feature(cart.feature.LAND, zorder=100, edgecolor='k')

        # Add a colorbar to the map
        cbar = plt.colorbar(sc, ax=ax, shrink=0.9)
        cbar.set_label('Temperature (°C)',fontsize=14,fontweight='bold')

        # Set the title and extent of the map
        ax.set_title('{}-{}'.format(year, month),fontsize=14,fontweight='bold')
        ax.set_extent(extent)

        # Show the plot
        plt.show()

        # Close the figure
        #plt.close(fig)

**Available date is from 1994/03 to 2022/12 (YYYY/MM)**<br>
**Therefore we should have total of 10533 days data but we only have for 6464 days**<br>
**1994/03 it has 306 days excluding January and February**<br>
**There are 7 leap years(1996,2000,2004,2008,2012,2016,2020)**<br>
**Total Calculation 1994/03 to 2022/12**<br>
  28 * 365 + 7(leap years) + 306 (from the year 1994) = 10220 + 7 +306 =10533 days <br>
  
**Total missing data (10533-6464)=4069 (38.63%) data is missing as per days**<br>
**We also have really sparse data**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the data into a pandas DataFrame
df = pd.read_csv('daily_averages.csv')

# Get the list of unique years in the data
years = df['year'].unique()

# Loop over the years and create subplots for each month within each year
for year in years:
    # Filter the data for the given year
    data = df[df['year'] == year]

    # Replace NaN temperature values with the mean temperature value
    mean_temp = data['Temperature'].mean()
    data['Temperature'] = data['Temperature'].fillna(mean_temp)

    # Get the list of unique months in the data for the given year
    months = data['month'].unique()

    # Create a new figure with a grid of subplots for each month
    fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(12, 8), sharex=True, sharey=True)

    # Flatten the 2D array of subplots into a 1D array for easier indexing
    axs = axs.flatten()

    # Loop over the months and plot the temperature data on the corresponding subplot
    for i, month in enumerate(months):
        # Filter the data for the given month
        month_data = data[data['month'] == month]

        # Get the corresponding subplot and plot the data
        ax = axs[i]
        ax.scatter(month_data['day'], month_data['Temperature'], color='darkgoldenrod', s=20)

        # Set the x-axis label and limits
        ax.set_xlabel('Day', fontsize=12, fontstyle='italic')
        ax.set_xticks(range(1, 32, 5))
        ax.set_xlim([0, 32])

        # Set the y-axis label
        ax.set_ylabel('Temperature', fontsize=14, fontstyle='italic',fontweight='bold')

        # Set the title of the subplot
        ax.set_title('{}-{}'.format(year, month), fontsize=14, fontweight='bold')

        # Add a legend to indicate what the markers represent
        #ax.legend(['Temperature'], loc='upper left', fontsize=10)
         # Add grid to the plot
        ax.grid(True)

    # Add a title for the entire plot
    fig.suptitle(str(year), fontsize=16, fontweight='bold')

    # Adjust the spacing between subplots
    fig.tight_layout()

    # Show the plot
    plt.show()